***UNIFICATION***

In [3]:
def unify(p1, p2):
    f1, f2 = p1.split("(")[0], p2.split("(")[0]
    if f1 != f2:
        return "Cannot be unified"

    a1, a2 = p1[p1.find("(")+1:-1].split(","), p2[p2.find("(")+1:-1].split(",")
    if len(a1) != len(a2):
        return "Cannot be unified"

    subs = {}
    for x, y in zip(a1, a2):
        x, y = x.strip(), y.strip()
        if x == y: continue
        if x.islower(): subs[x] = y
        elif y.islower(): subs[y] = x
        else: return "Cannot be unified"

    return subs or "Already unified"

print("Rule: Eligible(x) → Scholarship(x)")
print("Fact: Eligible(Ana)")
print("Substitution set:", unify("Eligible(x)", "Eligible(Ana)"))

Rule: Eligible(x) → Scholarship(x)
Fact: Eligible(Ana)
Substitution set: {'x': 'Ana'}


***FORWARD CHAINING***

In [4]:
facts = {
    "HighGrades(Juan)", "Applied(Juan)",
    "HighGrades(Maria)",  # DidNotApply(Maria) -> so Applied is absent
    "StudentLeader(Carlos)",  # NoHighGrades(Carlos) -> so HighGrades is absent
    "HighGrades(Ana)", "StudentLeader(Ana)", "Applied(Ana)"
}

# Rules as functions
def apply_rules(facts):
    derived = set(facts)
    changed = True
    while changed:
        changed = False

        # Rule 1: HG & Leader & Applied -> Eligible
        for f in list(derived):
            names = {f[f.find("(")+1:-1] for f in derived}
        for n in names:
            if all(pred+f"({n})" in derived for pred in ["HighGrades", "StudentLeader", "Applied"]):
                if f"Eligible({n})" not in derived:
                    derived.add(f"Eligible({n})")
                    changed = True

        # Rule 2: Eligible -> Scholarship
        for f in list(derived):
            if f.startswith("Eligible("):
                name = f[f.find("(")+1:-1]
                if f"Scholarship({name})" not in derived:
                    derived.add(f"Scholarship({name})")
                    changed = True

        # Rule 3: Scholarship -> FinancialSupport
        for f in list(derived):
            if f.startswith("Scholarship("):
                name = f[f.find("(")+1:-1]
                if f"FinancialSupport({name})" not in derived:
                    derived.add(f"FinancialSupport({name})")
                    changed = True
    return derived

final_facts = apply_rules(facts)

print("All derived facts:", final_facts)
print("Juan derived facts:", [f for f in final_facts if "Juan" in f])
print("Ana derived facts:", [f for f in final_facts if "Ana" in f])
print("Maria and Carlos not eligible because:")
print("- Maria did not Apply")
print("- Carlos has NoHighGrades")


=== Part 2: Forward Chaining ===
All derived facts: {'FinancialSupport(Ana)', 'HighGrades(Juan)', 'HighGrades(Ana)', 'Scholarship(Ana)', 'Applied(Ana)', 'StudentLeader(Ana)', 'Eligible(Ana)', 'Applied(Juan)', 'StudentLeader(Carlos)', 'HighGrades(Maria)'}
Juan derived facts: ['HighGrades(Juan)', 'Applied(Juan)']
Ana derived facts: ['FinancialSupport(Ana)', 'HighGrades(Ana)', 'Scholarship(Ana)', 'Applied(Ana)', 'StudentLeader(Ana)', 'Eligible(Ana)']
Maria and Carlos not eligible because:
- Maria did not Apply
- Carlos has NoHighGrades


***BACKWARD*** ***CHAINING***

In [5]:
def prove(goal, facts):
    # Base case: if goal is already a fact
    if goal in facts:
        return True

    name = goal[goal.find("(")+1:-1]

    # Scholarship(x) <- Eligible(x)
    if goal.startswith("Scholarship("):
        return prove(f"Eligible({name})", facts)

    # FinancialSupport(x) <- Scholarship(x)
    if goal.startswith("FinancialSupport("):
        return prove(f"Scholarship({name})", facts)

    # Eligible(x) <- HG(x) & Leader(x) & Applied(x)
    if goal.startswith("Eligible("):
        return all(p in facts for p in [
            f"HighGrades({name})",
            f"StudentLeader({name})",
            f"Applied({name})"
        ])
    return False

print("Goal Scholarship(Maria):", prove("Scholarship(Maria)", facts))
print("Goal FinancialSupport(Ana):", prove("FinancialSupport(Ana)", facts))

Goal Scholarship(Maria): False
Goal FinancialSupport(Ana): True


***VALIDITY*** ***CHECK***

In [8]:
import itertools
print("HG | L | A | Antecedent | Eligible | (Antecedent -> Eligible)")

for HG, L, A in itertools.product([0,1],[0,1],[0,1]):
    antecedent = HG and L and A
    Eligible = antecedent
    implication = (not antecedent) or Eligible
    print(f" {HG} | {L} | {A} |     {antecedent}      |    {Eligible}    |          {implication}")

print("\nStudent Tests:")
students = {
    "Juan": (1,0,1),
    "Maria": (1,0,0),
    "Carlos": (0,1,0),
    "Ana": (1,1,1)
}

for s,(HG,L,A) in students.items():
    antecedent = HG and L and A
    Eligible = antecedent
    status = "Eligible" if Eligible else "Not Eligible"
    print(f"{s}: HG={HG}, L={L}, A={A} → {status}")

HG | L | A | Antecedent | Eligible | (Antecedent -> Eligible)
 0 | 0 | 0 |     0      |    0    |          True
 0 | 0 | 1 |     0      |    0    |          True
 0 | 1 | 0 |     0      |    0    |          True
 0 | 1 | 1 |     0      |    0    |          True
 1 | 0 | 0 |     0      |    0    |          True
 1 | 0 | 1 |     0      |    0    |          True
 1 | 1 | 0 |     0      |    0    |          True
 1 | 1 | 1 |     1      |    1    |          1

Student Tests:
Juan: HG=1, L=0, A=1 → Not Eligible
Maria: HG=1, L=0, A=0 → Not Eligible
Carlos: HG=0, L=1, A=0 → Not Eligible
Ana: HG=1, L=1, A=1 → Eligible


Logical reasoning is important because it helps AI make fair and correct decisions by following clear rules. In real life, it’s also useful for us since it guides us to think step by step before deciding. This avoids mistakes and makes our choices more reliable. Just like in the scholarship case, logic helps decide who really deserves the support.